In [1]:
import pylab
import calendar
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
processed_data=pd.read_json("../data/processed_train.json")

In [2]:
random.seed(0)
train_data=processed_data.sample(n=processed_data.shape[0]*7/10)
test_data=processed_data.drop(train_data.index)

In [3]:
##########################first try with all numeric features in datasets##########################################
train=train_data.drop(['building_id','created','description','display_address','manager count','manager_id','listing_id','photos','street_address','UNIQUES','features'],axis=1)
test=test_data.drop(['building_id','created','description','display_address','manager count','manager_id','listing_id','photos','street_address','UNIQUES','features'],axis=1)

In [4]:
y_train=train.loc[:,'interest_level']
x_train=train.drop('interest_level',axis=1)
y_test=test.loc[:,'interest_level']
x_test=test.drop('interest_level',axis=1)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = DecisionTreeClassifier(max_depth=4)
clf5 = SVC(decision_function_shape='ovr',probability=True)
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('dt',clf4),('svc',clf5)], voting='soft')

params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200],}

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(x_train, y_train)

In [9]:
sum(grid.predict(x_train)==y_train)*1.0/y_train.shape[0]

0.85535228391130669

In [10]:
sum(grid.predict(x_test)==y_test)*1.0/y_test.shape[0]

0.71430501148183168

In [22]:
###using Logistic Reg, Random Forest, GaussianNB, DecisionTreeClassifier(bagging), KNN, SVC###
